### This is the program for searching a particular word among several file present in directory and sub-directories. This program takes these files as a input and returns the hightlighted word in a _'.docx'_ word file.
### Currently this program is tested on .txt, .dat and .docx files. And the sample words can be (case sensitive) :
* Apollo
* Cricket
* Bird
* Dog
* Machine Learning

In [ ]:
# Install
# !pip3 install python-docx 

In [1]:
import os
import docx
from docx import Document
from numpy import loadtxt
from docx.enum.text import WD_COLOR_INDEX

In [2]:
input_dir_path = "./input_files/" # Input Path
output_dir_path = "./output_files/"  # Output Path
file_type = ['.txt', '.docx' ,'.dat'] # Give the file exren
search_word = "Apollo" # Word to be searched - case sensitive

In [3]:
class file_reader:
    """
    The methods in the class is used to read the diffferent types of files.
    Currently it has two methods that are used to read the .txt, .dat and .docx files.
    """

    def __init__(self):
        pass
    
    # Read text file
    def read_txt_file(self, file_path):   
        text_file = open(file_path, "r")
        lines = text_file.read()
        return lines
    
    # Read docx file
    def read_docx_file(self, file_path):
        docment = docx.Document(file_path)
        fullText = []
        for para in docment.paragraphs:
            fullText.append(para.text)
        return ' '.join(fullText)
        

# This function is used to read the files and files from directory and sub directory.
def find_files_in_dir_and_subdir(path, path_list = [], sub_folders = True):
        try:   # Raising permission exception or in case file is not readable
            for entry in os.scandir(path):
                if entry.is_file():
                    path_list.append(entry.path)
                elif entry.is_dir() and sub_folders:   # if its a directory, then repeat process with recursive function
                    path_list = find_files_in_dir_and_subdir(entry.path,  path_list, sub_folders)
        except OSError:
            print('Cannot access ' + path +' Probably a permissions error')

        return path_list
        
def make_temp_doc(text):
    my_doc = Document()
    my_para = my_doc.add_paragraph(text)
    my_doc.save('temp.docx')

# This function is used to read the input text and highlight the word
def text_hightlighter(temp_docx, saving_docx_file_path, highlight_text):
    doc = Document(temp_docx)
    doc2 = Document()
    
    # Get the first paragraph's text
    p1_text = doc.paragraphs[0].text

    # Create a new paragraph with highlighted word
    p2 = doc2.add_paragraph()
    
    substrings = p1_text.split(highlight_text)
    
    for substring in substrings:
        p2.add_run(substring)
        font = p2.add_run(highlight_text).font
        font.highlight_color = WD_COLOR_INDEX.YELLOW # Give Colour to be highlighted
    p2.add_run(substrings[-1])

    # Save document under new name
    saving_docx_file_path = saving_docx_file_path + '.docx'
    doc2.save(saving_docx_file_path)
    # Remove the temp.docx file
    !rm -rf temp.docx


In [4]:


file_reader_obj = file_reader() # Call the file reader class

# Read all the files
file_list = find_files_in_dir_and_subdir(input_dir_path)


for file in file_list:
#     print(file)
    # Check if it is a proper file or not
    if any(file.endswith(ext) for ext in file_type):
        
        
        output_file_name = file.split('.')[1] 
        output_path_and_name = output_dir_path+output_file_name.split('/')[-1]
        
        if file.endswith('.txt'):
            text = file_reader_obj.read_txt_file(file)
            if search_word in text:
                make_temp_doc(text)
                text_hightlighter('temp.docx', output_path_and_name ,search_word)
                
        elif file.endswith('.docx'):
            docx_text = file_reader_obj.read_docx_file(file)
            if search_word in docx_text:
                make_temp_doc(docx_text)
                text_hightlighter('temp.docx', output_path_and_name ,search_word)